# Description
<div align="center">
    <img src="https://occ-0-8407-2219.1.nflxso.net/dnm/api/v6/E8vDc_W8CLv7-yMQu8KMEC7Rrr8/AAAABRa2Ojolth_4mw7fNztaPZJHe6V0oyMCD-NF0By0ddFyNQRb7NdWoHl3BPwSXVS3dbgG-jeiIaKzqAxqcpR_BGOHghInV3iHy6pY.jpg?r=f43" alt="The Crown" width="300"/>
</div>

A complex geometry has been constructed and meshed in SALOME and the resulting .med file is located in the `challenge_D` folder. You will need to first convert this med file to .xdmf, the file format readable by FEniCS, a function has been provided for this, which will also produce the correspondance dictionary, providing the ids for each tagged volume and surface.

The geometry has only one volume subdomain but several surfaces.

The material of the problem is Gold.

The temperature field is to be solved within FESTIM. Consider a thermal conductivity $315 \ \mathrm{W \ m^{-1}\  K^{-1}}$ and boundary conditions of a fixed temperature of $600 \ \mathrm{K}$ on $\Gamma_{\text{inner boundary}}$ and a convective heat flux on $\Gamma_{\text{outer boundary}}$, with a heat transfer coefficient of $10^{4} \ \mathrm{W \ m^{-2} \ K^{-1}}$ and outer temperature of $300\ \text{K}$.

For the hydrogen transport problem, consider an implantation flux of $10^{19} \ \mathrm{m^{2} s^{-1}}$ with an implantation depth of $3 \ \mathrm{nm}$ on $\Gamma_{\text{inner boundary}}$ and a fixed concentration of $0$ on $\Gamma_{\text{outer boundary}}$.

**📖 Tasks:**

- Evaluate the time to reach 99% of the equilibrium
- Evaluate the outwards flux
- Evaluate the hydrogen inventory at steady state
- Plot the temporal evolution of the outwards flux (on $\Gamma_{\text{outer boundary}}$) and hydrogen inventory
- Plot steady state temperature and mobile concentration fields
- Bonus: try and inspect the generated XDMF mesh files in Paraview

The transport properties for the geometry's material can be found below:

In [ ]:
import h_transport_materials as htm

D_gold = htm.diffusivities.filter(material="gold")[0]

print(f"Diffusivity of material: \n {D_gold}")

**💡Hint**: to visualise the fields produced by FESTIM, you can either export to XDMF using the ``XDMFExport`` class and then open the files in Paraview. Alternatively, you can visualise it in python with the ``fenics.plot`` function:

```python
import fenics as f
import festim as F
import matplotlib.pyplot as plt

my_model = F.Simulation(.....)
my_model.initialise()
my_model.run()

plt.figure()
c_mobile = my_model.h_transport_problem.mobile.mobile_concentration()
f.plot(c_mobile)

plt.figure()
T = my_model.T.T
f.plot(T)
```

In [ ]:
import meshio


def convert_med_to_xdmf(
    med_file,
    cell_file="mesh_domains.xdmf",
    facet_file="mesh_boundaries.xdmf",
    cell_type="tetra",
    facet_type="triangle",
):
    """Converts a MED mesh to XDMF
    Args:
        med_file (str): the name of the MED file
        cell_file (str, optional): the name of the file containing the
            volume markers. Defaults to "mesh_domains.xdmf".
        facet_file (str, optional): the name of the file containing the
            surface markers.. Defaults to "mesh_boundaries.xdmf".
        cell_type (str, optional): The topology of the cells. Defaults to "tetra".
        facet_type (str, optional): The topology of the facets. Defaults to "triangle".
    Returns:
        dict, dict: the correspondance dict, the cell types
    """
    msh = meshio.read(med_file)

    correspondance_dict = msh.cell_tags

    cell_data_types = msh.cell_data_dict["cell_tags"].keys()

    for mesh_block in msh.cells:
        if mesh_block.type == cell_type:

            meshio.write_points_cells(
                cell_file,
                msh.points,
                [mesh_block],
                cell_data={"f": [-1 * msh.cell_data_dict["cell_tags"][cell_type]]},
            )
        elif mesh_block.type == facet_type:
            meshio.write_points_cells(
                facet_file,
                msh.points,
                [mesh_block],
                cell_data={"f": [-1 * msh.cell_data_dict["cell_tags"][facet_type]]},
            )

    return correspondance_dict, cell_data_types


mesh_file = "challenge_D/challenge_mesh.med"

correspondance_dict, cell_data_types = convert_med_to_xdmf(
    mesh_file,
    cell_type="triangle",
    facet_type="line",
    cell_file="challenge_D/mesh_domains.xdmf",
    facet_file="challenge_D/mesh_boundaries.xdmf",
)

print("This is the correspondance dict:")
print(correspondance_dict)

## Your answer

In [ ]:
import festim as F

# YOUR CODE GOES HERE ...
